# Lectura 32: Lazy API

Vamos a crear una consulta lazy a partir del dataset de vuelos y aplicaremos algunas transformaciones. 

Al iniciar la consulta con `pl.scan_parquet`, estamos utilizando la API lazy.

## Uso de la API lazy a partir de la lectura de un archivo

In [ ]:
import polars as pl
from polars import col

query = (
    pl.scan_parquet('./data/vuelos/vuelos.parquet').with_columns(
        col('AIR_TIME') / 60
    )
    .filter(col('MONTH') > 10)
)

type(query)

En esta consulta le decimos a Polars que queremos:

- cargar datos desde el archivo `vuelos.parquet`
- convertir la columna `AIR_TIME` a horas suponiendo que lo que contiene son minutos
- aplicar un filtro a la columna `MONTH`

La consulta lazy no se ejecutará en este momento a menos que le apliquemos un `.collect()` .

In [ ]:
query.collect()

## Uso de la API lazy desde un DataFrame

Una forma alternativa de acceder a la API lazy es llamar a `.lazy` en un DataFrame que ya se ha creado en la memoria.

Al llamar a `.lazy` convertimos el DataFrame en un LazyFrame.

In [ ]:
df = pl.DataFrame(
    {
        'id': [1, 2, 3],
        'nombre': ['Rosa', 'Ana', 'Pedro']
    }
)

df_lazy = df.lazy()

In [ ]:
type(df_lazy)

## Schema

El esquema de un DataFrame o un LazyFrame establece los nombres de las columnas y sus tipos de datos. Si recordamos podemos ver el esquema de un DataFrame o un LazyFrame con el método `.schema`.

In [ ]:
df.schema

In [ ]:
df_lazy.schema

## Comprobación de tipos de datos en la API lazy

El esquema juega un papel importante en la API lazy. 

Una ventaja de la API lazy es que Polars comprobará el esquema antes de procesar cualquier dato. Esta verificación ocurre cuando ejecutamos nuestra consulta lazy.

Vemos cómo funciona esto en el siguiente ejemplo donde llamamos a la expresión `.round` en la columna `id` del DataFrame que previamente hemos creado.

In [ ]:
df_lazy.with_columns(col('id').round(0)).collect()

Los que ha sucedido es que la expresión `.round` solo es válida para columnas de tipo float. Llamar a `.round` en una columna de números enteros significa que la operación generará un  error de tipo `InvalidOperationError` cuando evaluemos la consulta con `collect`. Esta verificación de esquema ocurre antes de que se procesen los datos cuando llamamos a `collect`.

## Manejo de operaciones no disponibles en la API lazy

In [ ]:
lazy_eager_query = (
    pl.scan_parquet('./data/vuelos/vuelos.parquet').with_columns(
        air_time_hrs=(col('AIR_TIME') / 60)
    )
    .collect()
    .pivot(
        index='AIRLINE', columns='MONTH', values='AIR_TIME', aggregate_function='mean'
    )
    .lazy()
    .filter(col('AIRLINE') == 'AA')
    .collect()
)

display(lazy_eager_query)